<a href="https://colab.research.google.com/github/nicha1997/Data-Analytics-for-Business/blob/main/LSTM_Model_for_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, Input

# Import data
file_path = r'D:\desktop\set.xlsx'  # Local file path
data = pd.read_excel(file_path)
dates = pd.to_datetime(data['date'])  # Convert the date to date format


# Define a function to create a dataset, considering leap years.
def create_dataset_for_single_item(dates, dataset, lookback_years=5):
    dataX, dataY = [], []
    # Assume data is only available up to 2017, and predict for 2018.
    start_2018_idx = len(dates)  # The data ends in 2017, assume the starting index for 2018 is the day after the last day.
    target_dates = pd.date_range(start='2018-01-01', periods=365)  # Generate dates for 2018.

    for i in range(365):  # Predict 365 days for 2018.
        x = []
        target_date = target_dates[i]  # Current date in 2018.

        for year in range(1, lookback_years + 1):
            current_date = target_date - pd.DateOffset(years=year)  # Trace back previous years.
            # Handle leap year cases: if the current date is February 29th and it is not a leap year, skip this date.
            if current_date.month == 2 and current_date.day == 29:
                if not ((current_date.year % 4 == 0 and current_date.year % 100 != 0) or (
                        current_date.year % 400 == 0)):
                    current_date = current_date - pd.DateOffset(days=1)  # Adjust to February 28th.

            current_date_idx = np.where(dates == current_date)[0]

            if len(current_date_idx) > 0:
                x.append(dataset[current_date_idx[0]])
        if len(x) == lookback_years:  # Ensure there is complete data for five years.
            dataX.append(x)
            dataY.append(dataset[start_2018_idx - 365 + i])  # Use the last 365 days of 2017 as labels.
    return np.array(dataX), np.array(dataY)


# Initialize the results DataFrame.
all_predictions = pd.DataFrame({'date': pd.date_range(start='2018-01-01', periods=365)})

# Iterate over all product columns for prediction.
for column in data.columns[1:]:  # Skip the 'date' column and predict the sales of each product.
    print(f"Predicting sales of {column}...")

    # Get sales data for the product.
    dataset = data[column].values

    # Normalize the data.
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_target = scaler.fit_transform(dataset.reshape(-1, 1))  # Normalize target data.

    # Use data from the same day between 2013-2017 as features.
    trainX, trainY = create_dataset_for_single_item(dates, scaled_target, lookback_years=5)

    # Build the LSTM model.
    input_shape = Input(shape=(trainX.shape[1], 1))  # Five years of data for each day.
    lstm1 = layers.LSTM(128, return_sequences=True)(input_shape)
    lstm2 = layers.LSTM(256)(lstm1)
    dense1 = layers.Dense(128, activation="relu")(lstm2)
    dropout = layers.Dropout(rate=0.2)(dense1)
    output_shape = layers.Dense(1)(dropout)

    # Define the model.
    lstm_model = tf.keras.Model(inputs=input_shape, outputs=output_shape)
    lstm_model.compile(loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                       metrics=["mse"])

    # Train the model.
    lstm_model.fit(trainX, trainY, batch_size=32, epochs=100, validation_split=0.2, verbose=1)

    # Predict data for 2018.
    item_predictions = []

    # Predict each day of 2018.
    for day in range(365):
        input_data = trainX[day].reshape(1, 5, 1)  # The sales data of this product over the past four years
        prediction = lstm_model.predict(input_data)

        # Denormalize the predicted values.
        prediction_real = scaler.inverse_transform(prediction)

        # Add the predicted values to the results.
        item_predictions.append(prediction_real[0][0])

    # Add the prediction results for this product to the DataFrame.
    all_predictions[column] = item_predictions

# Save all prediction results as an Excel file.
output_file_path = r'D:\desktop\all_items_predictions_2018.xlsx'
all_predictions.to_excel(output_file_path, index=False)
print(f"The forecast results for all products have been saved as an Excel file:{output_file_path}")